# ДЗ1 
#### Реализация модельного дерева решений с линейной регрессией в листьях (задача регрессии)
##### Копин Борис Александровч

http://stats.stackexchange.com/questions/78563/regression-tree-algorithm-with-linear-regression-models-in-each-leaf
http://stackoverflow.com/questions/11810949/difference-between-regression-tree-and-model-tree
http://stats.stackexchange.com/questions/168964/building-a-regression-tree-with-r-from-scratch
http://stackoverflow.com/questions/23986164/is-tree-decisiontreeregressor-a-model-tree-or-a-regression-tree

[CART](https://github.com/yan9yu/python-books/blob/master/Machine%20Learning/%5BO'Reilly%5D%20-%20Programming%20Collective%20Intelligence.pdf)

- Model trees - http://sci2s.ugr.es/keel/pdf/algorithm/congreso/1992-Quinlan-AI.pdf
- - http://download.springer.com/static/pdf/269/art%253A10.1023%252FA%253A1007421302149.pdf?originUrl=http%3A%2F%2Flink.springer.com%2Farticle%2F10.1023%2FA%3A1007421302149&token2=exp=1455987988~acl=%2Fstatic%2Fpdf%2F269%2Fart%25253A10.1023%25252FA%25253A1007421302149.pdf%3ForiginUrl%3Dhttp%253A%252F%252Flink.springer.com%252Farticle%252F10.1023%252FA%253A1007421302149*~hmac=d87bfe4b77411a48be24de01de791f2cbf99719ee90a50acd1bc510697804e39
- Logistic Model Trees - http://www.cs.waikato.ac.nz/~eibe/pubs/LandwehrHallFrankCameraReady.pdf

http://scikit-learn.org/stable/modules/tree.html#tree

In [247]:
import pandas as pd

In [248]:
import numpy as np

In [249]:
from collections import defaultdict
a = defaultdict(int)

In [250]:
class decisionnode:
    def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
        self.col=col
        self.value=value
        self.results=results
        self.tb=tb
        self.fb=fb

In [251]:
# TODO: this can be done by pandas builtin functions
# Create counts of possible results (the last column of
# each row is the result)
def uniquecounts(rows):
    results={}
    for row in rows:
        # The result is the last column
        r = row[len(row)-1]
        if r not in results:
            results[r]=0
        results[r]+=1
    return results

In [252]:
# Divides a set on a specific column. Can handle numeric
# or nominal values
def divideset(rows, column, value):
    # Make a function that tells us if a row is in
    # the first group (true) or the second group (false)
    split_function = None
    if isinstance(value, int) or isinstance(value, float):
        split_function = lambda row:row[column] >= value
    else:
        split_function = lambda row:row[column] == value

    # Divide the rows into two sets and return them
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return (set1, set2)

In [253]:
# Entropy is the sum of p(x)log(p(x)) across all
# the different possible results
def entropy(rows):
    from math import log
    log2=lambda x:log(x)/log(2)
    results=uniquecounts(rows) # Now calculate the entropy
    ent=0.0

    for r in results.keys():
        p=float(results[r])/len(rows)
        ent=ent-p*log2(p)

    return ent

In [254]:
def regscore(rows):
    return np.std([row[-1] for row in rows])    

# The first builds an ordinary decision tree, using as splitting criterion the maximization of the intra-subset variation of the target value

In [255]:
def buildtree(rows, scoref=entropy):
    if len(rows)==0:
        return decisionnode()
    
    current_score = scoref(rows)
    # Set up some variables to track the best criteria
    best_gain = 0.0
    best_criteria = None
    best_sets = None
        
    column_count = len(rows[0])-1
    for col in range(0,column_count):
        # Generate the list of different values in
        # this column
        column_values={}
        for row in rows:
            column_values[row[col]]=1

        # Now try dividing the rows up for each value
        # in this column
        for value in column_values.keys():
            (set1, set2)=divideset(rows, col, value)

            # Information gain
            p = float(len(set1))/len(rows)
            gain = current_score - p*scoref(set1)-(1-p)*scoref(set2)

            if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1,set2)

    # Create the subbranches
    if best_gain > 0:
        trueBranch=buildtree(best_sets[0])
        falseBranch=buildtree(best_sets[1])
        return decisionnode(col=best_criteria[0],
                            value=best_criteria[1],
                            tb=trueBranch,
                            fb=falseBranch)
    else:
        return decisionnode(results=uniquecounts(rows))
                
                

In [256]:
x = np.array([
    [0,0],
    [1, 1],
    [2, 2],
    [3, 3]
])
y = np.array([-1, 0.2, 0.9, 2.1])


X = np.vstack([np.ones(x.shape[0]), x.T]).T
print X
print X.shape
m = np.linalg.lstsq(X, y)[0]
print m


[[ 1.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  2.  2.]
 [ 1.  3.  3.]]
(4, 3)
[-0.95  0.5   0.5 ]


In [257]:
def build_model_tree(rows, scoref=entropy):
    if len(rows)==0:
        return decisionnode()
    
    current_score = scoref(rows)
    # Set up some variables to track the best criteria
    best_gain = 0.0
    best_criteria = None
    best_sets = None
        
    column_count = len(rows[0])-1
    for col in range(0,column_count):
        # Generate the list of different values in
        # this column
        column_values={}
        for row in rows:
            column_values[row[col]]=1

        # Now try dividing the rows up for each value
        # in this column
        for value in column_values.keys():
            (set1, set2)=divideset(rows, col, value)

            # Information gain
            p = float(len(set1))/len(rows)
            gain = current_score - p*scoref(set1)-(1-p)*scoref(set2)

            if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1,set2)

    # Create the subbranches
    if best_gain > 0:
        trueBranch=buildtree(best_sets[0])
        falseBranch=buildtree(best_sets[1])
        return decisionnode(col=best_criteria[0],
                            value=best_criteria[1],
                            tb=trueBranch,
                            fb=falseBranch)
    else:
        return decisionnode(results=uniquecounts(rows))
                
                

In [258]:
data = [
    ['slashdot', 'USA', 'yes', 18, 'None', 120],
    ['google', 'France', 'yes', 23, 'Premium', 130],
    ['google', 'France', 'yes', 23, 'Premium', 140],
    ['google', 'France', 'yes', 23, 'Premium', 120],
    ['google', 'France', 'yes', 23, 'Premium', 110],
]

In [259]:
tree = buildtree(data, scoref=regscore)

In [260]:
tree

<__main__.decisionnode instance at 0x1113f9cf8>

In [261]:
class decisionnode:
    def __init__(self, col=-1, value=None, results=None, tb=None, fb=None):
        self.col=col
        self.value=value
        self.results=results
        self.tb=tb
        self.fb=fb

In [262]:
divideset(data, 1, 'USA')

([['slashdot', 'USA', 'yes', 18, 'None', 120]],
 [['google', 'France', 'yes', 23, 'Premium', 130],
  ['google', 'France', 'yes', 23, 'Premium', 140],
  ['google', 'France', 'yes', 23, 'Premium', 120],
  ['google', 'France', 'yes', 23, 'Premium', 110]])

In [263]:
len(data[0])

6

In [264]:
w = np.dot(np.dot(np.dot(X_train.T, X_train), X_train.T), y_train)

In [266]:
X_test[0]

array([  10.8342,    0.    ,   18.1   ,    0.    ,    0.679 ,    6.782 ,
         90.8   ,    1.8195,   24.    ,  666.    ,   20.2   ,   21.57  ,
         25.79  ])

In [1]:
from collections import Counter

In [10]:
from collections import Counter
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn import cross_validation as cv
from sklearn.metrics import mean_squared_error

In [11]:
housing = datasets.load_boston()

In [23]:
df = pd.read_csv("./spam.train.txt", delim_whitespace=True, header=None)
y = df.ix[:, 0]
X = df.ix[:, 1:]

In [24]:
X_train, X_test, y_train, y_test = cv.train_test_split(X.as_matrix(),
                                                       y.as_matrix(),
                                                       test_size=0.25,
                                                       random_state=288)

In [25]:
# INTERNAL
# LEAF

class Node(object):
    def __init__(self, X, y,
                 feature_ind=None, value=None,
                 tb=None, fb=None, build_model=False):
        self.X = X
        self.y = y
        self.tb = tb
        self.fb = fb
        
        self.feature_ind = feature_ind
        self.value = value
        
        self.left = None
        self.right = None
        
        self.weights = None
        self.features_ind = set([])
        
#         if build_model:
#             self.fit()
        
    def fit(self):
        self.weights = ols(self.X[:, list(self.features_ind)], self.y)
#         if len(self.features_ind):
#             self.weights = ols(self.X[:, list(self.features_ind)], self.y)
#         else:
#             self.weights = ols(self.X, self.y)
    
    def predict(self, X_pred):
        if len(self.features_ind):
            return np.dot(X_pred[:, list(self.features_ind)], self.weights)
        else:
            return np.mean(self.y)
        
#         if len(self.features_ind):
#             return np.dot(X_pred[:, list(self.features_ind)], self.weights)
#         print self.weights
#         return np.dot(X_pred, self.weights)

    @staticmethod
    def _error(node):
        n = node.X.shape[0]
        v = len(node.weights)
        deviation_mean = np.sum(np.abs(node.predict(node.X) - node.y))/float(n)
        params_norm = (n+v)/(n-v)
        return params_norm*deviation
        
#     @staticmethod
#     def _subtree_error(tree):
#         if tree.feature_ind:
#         else:

- Build default tree
- Build linear regression in default tree
- Estimate error in default tree
- Prune tree

In [26]:
def ols(X, y):
    inv = np.linalg.inv(np.dot(X.T, X))
    return np.dot(np.dot(inv, X.T), y)

In [27]:
class m5(object):
    def __init__(self, fit_intercept=True, min_samples_leaf=4, sd_ratio=0.05, using_m5=True):
        self.fit_intercept = fit_intercept
        self.min_samples_leaf = min_samples_leaf
        self.sd_ratio = sd_ratio
        self.using_m5 = using_m5
        
        self.tree = None
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.hstack((np.ones((X.shape[0], 1), dtype=X.dtype), X))
        # ToDo: Convert nominal to binary
        
        self.SD = np.std(y)
        
        self.default_tree = self._build_default_tree(X, y)
        
#         if self.using_m5:
        self.model_tree = self._prune_tree(self.default_tree)
        
        return self.default_tree
    
    def predict(self, X):
        if self.fit_intercept:
            X = np.hstack((np.ones((X.shape[0], 1), dtype=X.dtype), X))
        return np.array([self._tree_predict(x, self.default_tree) for x in X])
        
    def _tree_predict(self, x, tree):
        if not tree.feature_ind:
            return tree.predict(x) # np.mean(tree.y)

        if x[tree.feature_ind] >= tree.value:
            if tree.tb:
                return self._tree_predict(x, tree.tb)
        else:
            if tree.fb:
                return self._tree_predict(x, tree.fb)

    def _build_default_tree(self, X, y):
        # Set up some variables to track the best criteria
        if X.shape[0] < 4 or np.std(y) < self.sd_ratio*self.SD:
            return Node(X, y)
    
        best_sdr = 0.0
        best_criteria = None
        best_split = None

        for feature_ind in range(X.shape[1]):
            # Generate the list of different values of
            # this feature
            attr_values = Counter(X[:, feature_ind])

            # Compute SDR(Standart Deviation Reduction) http://take.ms/TeWHJ
            # for each attr_value
            for attr_value in attr_values.keys():
                split_ind = X[:, feature_ind] >= attr_value

                X_T1, y_T1 = X[split_ind], y[split_ind]
                p_T1 = len(y_T1)/float(len(y))

                X_T2, y_T2 = X[np.invert(split_ind)], y[np.invert(split_ind)]
                p_T2 = len(y_T2)/float(len(y))

                sdr = np.std(y) - p_T1*np.std(y_T1) - p_T2*np.std(y_T2)

                if sdr > best_sdr and len(y_T1) > 0 and len(y_T2) > 0:
                    best_sdr = sdr
                    best_criteria = (feature_ind, attr_value)
                    best_split = ((X_T1, y_T1), (X_T2, y_T2))
                    
        T1, T2 = best_split

        size_cond = len(T1[1]) < self.min_samples_leaf or len(T2[1]) < self.min_samples_leaf
        std_cond = np.std(T1[1]) < self.sd_ratio*self.SD or np.std(T2[1]) < self.sd_ratio*self.SD
        
        if size_cond or std_cond or best_sdr <= 0: # Leaf node
            return Node(X, y)
        else: # Create the subbranches
#             print "l", len(T1[0]), "r", len(T2[0])

            true_branch = self._build_default_tree(*T1)
            false_branch = self._build_default_tree(*T2)
            return Node(X, y,
                        best_criteria[0], best_criteria[1],
                        true_branch, false_branch)

    def _prune_tree(self, tree):
        self._collect_features(tree)
        return tree
        
    def _collect_features(self, tree):
        if not tree.feature_ind:
            return set([])

        tree.features_ind = self._collect_features(tree.tb) | tree.features_ind
        tree.features_ind = self._collect_features(tree.fb) | tree.features_ind

        return tree.features_ind | set([tree.feature_ind])

    def _subtree_error(self):
        raise NotImplementedError

In [28]:
regressor = m5(using_m5=True)
tree = regressor.fit(X_train, y_train)

In [29]:
y_pred = regressor.predict(X_test)

In [30]:
mean_squared_error(y_test, y_pred)

0.064791941967992178

In [33]:
from sklearn.tree import DecisionTreeRegressor


dtr = DecisionTreeRegressor(min_samples_leaf=4)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
mean_squared_error(y_test, y_pred)


0.062061657808557477